In [1]:
# learning systems project 2

In [1]:
#variables:
#name of the node, a count
#nodelink used to link similar items
#parent vaiable used to refer to the parent of the node in the tree
#node contains an empty dictionary for the children in the node
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
rootNode = treeNode('pyramid',9,None)

In [3]:
rootNode.children['eye'] = treeNode('eye',13,None)

In [5]:
rootNode.disp()

   pyramid   9
     eye   13


In [46]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [47]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [9]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [59]:
import numpy as np
from collections import Counter
import pandas as pd
import os
import PIL.Image

In [82]:
data = pd.read_csv("out.csv")

In [83]:
l = []

In [84]:
df=pd.DataFrame(data)
for row in df.iterrows():
    index, data = row
    l.append(data.tolist())

In [86]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [85]:
l

[[1002945, '*', '4', 4, 5, 7, '10', 3, 2, 1, 2],
 [1015425, '3', '1', 1, 1, 2, '2', 3, 1, 1, 2],
 [1016277, '*', '*', 8, 1, 3, '4', 3, 7, 1, 2],
 [1017023, '4', '1', 1, 3, 2, '1', 3, 1, 1, 2],
 [1017122, '*', '*', 10, 8, 7, '10', 9, 7, 1, 4],
 [1018099, '1', '1', 1, 1, 2, '10', 3, 1, 1, 2],
 [1018561, '2', '1', 2, 1, 2, '1', 3, 1, 1, 2],
 [1033078, '2', '1', 1, 1, 2, '1', 1, 1, 5, 2],
 [1033078, '4', '2', 1, 1, 2, '1', 2, 1, 1, 2],
 [1035283, '1', '1', 1, 1, 1, '1', 3, 1, 1, 2],
 [1036172, '2', '1', 1, 1, 2, '1', 2, 1, 1, 2],
 [1041801, '*', '3', 3, 3, 2, '3', 4, 4, 1, 4],
 [1043999, '1', '1', 1, 1, 2, '3', 3, 1, 1, 2],
 [1044572, '*', '*', 5, 10, 7, '9', 5, 5, 4, 4],
 [1047630, '*', '4', 6, 4, 6, '1', 4, 3, 1, 4],
 [1048672, '4', '1', 1, 1, 2, '1', 2, 1, 1, 2],
 [1049815, '4', '1', 1, 1, 2, '1', 3, 1, 1, 2],
 [1050670, '*', '*', 7, 6, 4, '10', 4, 1, 2, 4],
 [1050718, '*', '1', 1, 1, 2, '1', 3, 1, 1, 2],
 [1054590, '*', '3', 2, 10, 5, '10', 5, 4, 4, 4],
 [1054593, '*', '*', 5, 3, 6, '7

In [75]:
temp = [['r', '*', '4', 'j', 1],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [87]:
feed = createInitSet(l)

In [88]:
feed

{frozenset({1002945, 2, 3, 4, 5, '4', '*', 7, '10', 1}): 1,
 frozenset({1015425, 2, 1, 3, '1', '3', '2'}): 1,
 frozenset({1, 2, 3, '4', '*', 8, 7, 1016277}): 1,
 frozenset({1, 2, 3, '4', '1', 1017023}): 1,
 frozenset({1, 1017122, 4, '10', 7, 8, '*', 10, 9}): 1,
 frozenset({1, 2, 3, '10', '1', 1018099}): 1,
 frozenset({1018561, 2, 1, 3, '1', '2'}): 1,
 frozenset({1, 2, 5, '1', 1033078, '2'}): 1,
 frozenset({1, 2, '4', '1', 1033078, '2'}): 1,
 frozenset({1, 2, 3, '1', 1035283}): 1,
 frozenset({1, 2, 1036172, '1', '2'}): 1,
 frozenset({1, 2, 3, 4, '*', 1041801, '3'}): 1,
 frozenset({1, 2, 3, '1', '3', 1043999}): 1,
 frozenset({4, 5, 7, '*', 10, '9', 1044572}): 1,
 frozenset({1, 3, 4, '4', 6, '*', '1', 1047630}): 1,
 frozenset({1048672, 1, 2, '4', '1'}): 1,
 frozenset({1, 2, 3, '4', '1', 1049815}): 1,
 frozenset({1, 2, 4, '10', 6, 7, '*', 1050670}): 1,
 frozenset({1, 2, 3, '*', '1', 1050718}): 1,
 frozenset({2, 4, 5, '10', '*', 10, '3', 1054590}): 1,
 frozenset({1054593, 1, 3, 4, 5, 6, '*'

In [89]:
myFPtree, myHeaderTab = createTree(feed, 3)

In [90]:
myFPtree.disp()

   Null Set   1
     1   599
       2   490
         *   49
           3   28
             4   24
               4   10
                 10   4
                   5   2
                     7   1
                     6   1
                   7   1
                     9   1
                   6   1
                 5   1
                   7   1
                     ?   1
                 10   3
                   2   1
                   6   1
                   10   1
                     7   1
                 9   1
               3   7
                 4   3
                   5   1
                     6   1
                 10   1
                   5   1
                     5   1
                 7   1
                   5   1
                 10   1
                   7   1
                     6   1
               2   1
                 6   1
                   7   1
               10   5
                 6   2
                 5   2
                 7   1
                   

In [91]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [92]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [93]:
findPrefixPath('r', myHeaderTab['4'][1])

{frozenset({1, 2, 3, '*'}): 2,
 frozenset({1, 2, 3, 4, '*'}): 10,
 frozenset({1, 2, '1'}): 35,
 frozenset({1, 2, 3, '1'}): 22,
 frozenset({4, '*'}): 1,
 frozenset({1, 4, '*'}): 11,
 frozenset({1, 3, 4, '*', '1'}): 1,
 frozenset({1, 2}): 1,
 frozenset({1, 2, 4, '*'}): 6,
 frozenset({1, 2, 3}): 1,
 frozenset({'3', 2, 4, 1}): 2,
 frozenset({1, 3, 4}): 1,
 frozenset({3, 4, '*'}): 3,
 frozenset({1, 2, 3, 4, '*', '3'}): 3,
 frozenset({'3', 2, '1', 1}): 3,
 frozenset({2, 4, '*'}): 2,
 frozenset({1, 3, 4, '1'}): 1,
 frozenset({1, 2, 3, 4}): 2,
 frozenset({1, 2, 4, '1'}): 1,
 frozenset({1, 2, 3, 4, '1'}): 1,
 frozenset({1, 3, 4, '*'}): 10,
 frozenset({2, 3, 4, '*'}): 4,
 frozenset({1, 2, 3, 4, '*', '1'}): 1,
 frozenset({1, 3, 4, '1', '3'}): 1,
 frozenset({1, 2, 3, 4, '3'}): 1,
 frozenset({1, 2, 3, '1', '3'}): 4,
 frozenset({1, 2, 4, '1', '3'}): 1,
 frozenset({1, 4, '1', '*'}): 2,
 frozenset({1, 2, 3, '*', '1'}): 1}